In [ ]:
!pip install -q -U keras-tuner

     |████████████████████████████████| 98 kB 2.8 MB/s 


In [ ]:
import tensorflow as tf
from tensorflow.keras.datasets import cifar10
from tensorflow import keras
from keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import SGD, Adadelta, Adam
from tensorflow.keras.layers import Conv2D, Dense, MaxPooling2D, Flatten, Dropout
import keras_tuner as kt

In [ ]:
(raw_x_train, raw_y_train), (raw_x_test, raw_y_test) = cifar10.load_data()

170508288/170498071 [==============================] - 3s 0us/step


In [ ]:
def rescale_data(train_x, test_x, train_y, test_y):
    train_x = train_x.astype('float32')
    test_x = test_x.astype('float32')
    train_x = train_x / 255.0
    test_x = test_x / 255.0
    train_y = to_categorical(train_y)
    test_y = to_categorical(test_y)
    return (train_x, train_y), (test_x, test_y)  

In [ ]:
def build_model(hp):
    model = Sequential()
    model.add(Conv2D(32, (3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=(32, 32, 3)))
    for i in range(hp.Int("num_layers", 1, 5)):
      filters = hp.Int(f"filters_{i}", min_value=32, max_value=256, step=32)
      model.add(Conv2D(filters, (3, 3), activation='relu', kernel_initializer='he_uniform'))
    model.add(MaxPooling2D((2, 2)))
    model.add(Flatten())
    for i in range(hp.Int("num_layers", 1, 5)):
      units = hp.Int(f"units_{i}", min_value=32, max_value=256, step=32)
      model.add(Dense(units=units, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(10, activation='softmax'))
    # compile model
    learning_rate = hp.Float("lr", min_value=1e-4, max_value=0.001, sampling="log")
    opt = Adam(learning_rate=learning_rate)
    model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])
    return model  

In [ ]:
(train_x, train_y), (test_x, test_y) = rescale_data(raw_x_train, raw_x_test, raw_y_train, raw_y_test)
model = build_model(kt.HyperParameters())

In [ ]:
tuner = kt.RandomSearch(
    hypermodel=build_model,
    objective="val_accuracy",
    max_trials=3,
    executions_per_trial=2,
    overwrite=True
)

In [ ]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 1, 'max_value': 5, 'step': 1, 'sampling': None}
filters_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 256, 'step': 32, 'sampling': None}
lr (Float)
{'default': 0.0001, 'conditions': [], 'min_value': 0.0001, 'max_value': 0.001, 'step': None, 'sampling': 'log'}


In [ ]:
tuner.search(train_x, train_y, epochs=10, validation_data=(test_x, test_y))

Trial 3 Complete [00h 08m 34s]
val_accuracy: 0.7081499993801117

Best val_accuracy So Far: 0.7081499993801117
Total elapsed time: 00h 27m 01s
INFO:tensorflow:Oracle triggered exit


In [ ]:
# Get the best 2 models.
models = tuner.get_best_models(num_models=2)
best_model = models[0]
best_model.build(input_shape=(32, 32, 3))
best_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 30, 30, 32)        896       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 64)        18496     
                                                                 
 conv2d_2 (Conv2D)           (None, 26, 26, 96)        55392     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 96)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 16224)             0         
                                                                 
 dense (Dense)               (None, 224)               3634400   
                                                        

In [ ]:
tuner.results_summary()

Results summary
Results in ./untitled_project
Showing 10 best trials
Objective(name='val_accuracy', direction='max')
Trial summary
Hyperparameters:
num_layers: 2
filters_0: 64
units_0: 224
lr: 0.0003783692348881492
filters_1: 96
units_1: 128
Score: 0.7081499993801117
Trial summary
Hyperparameters:
num_layers: 2
filters_0: 224
units_0: 32
lr: 0.0007615764539410149
filters_1: 32
units_1: 32
Score: 0.696150004863739
Trial summary
Hyperparameters:
num_layers: 1
filters_0: 128
units_0: 96
lr: 0.0001212404390170712
filters_1: 32
units_1: 96
Score: 0.6661500036716461
